## BLS Usual Weekly Earnings Data and CPS equivalent

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Series stored as a dictionary
series = {'LEU0252911200': 'p10uwe',
          'LEU0252911300': 'p25uwe',
          'LEU0252881500': 'p50uwe',
          'LEU0252911400': 'p75uwe',
          'LEU0252911500': 'p90uwe',
          'LEU0254466800': 'nuwe'}

# Start year and end year
dates = (1988, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'uwe_main.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### UWE and Growth

In [3]:
# Retrieve relavant CPS data
columns = ['MONTH', 'YEAR', 'AGE', 'PWORWGT', 'WKEARN', 
           'HRSUSL1', 'WORKFT']
dfc = (pd.concat([pd.read_feather(cps_dir / f'cps{year}.ft', 
                                  columns=columns)
                    .query('WKEARN > 0 and WORKFT == 1')
                  for year in range(1989, 2024)]))

data = pd.DataFrame()
# Calculate wage for percentile p
for p in [10, 25, 50, 75, 90]:
    data[f'p{p}'] = (dfc.groupby(['YEAR', 'MONTH'])
                     .apply(lambda x: binned_wage(x, percentile=p/100)))
data.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') 
              for ti in data.index]

# Calculate 3 month moving average and 12-month growth rates
for col in data.columns:
    data[f'{col}_3M'] = data[col].rolling(3).mean()
for col in data.columns:
    data[f'{col}_gr'] = data[col].pct_change(12) * 100
    
# Save to csv
data.to_csv(data_dir / 'uwe_cps.csv', index_label='date')    
    
# Labels for bar chart
data['label'] = [dt.strftime('%b\\\%Y') if dt.month == 1 
                 else dt.strftime('%b') for dt in data.index]
nextmo = data.index[-1] + pd.DateOffset(months=1)
data.loc[nextmo, 'label'] = ''
data['FILL'] = 0
cut = -9
if len(data.label.iloc[cut]) < 5:
    year = data.index[cut].year
    data.loc[data.index[cut], 'label'] = (data.label.iloc[cut] + 
                                         f'\\\\{data.index[cut].year}')
data.iloc[cut:].to_csv(data_dir / 'uwe_cps_sh.csv', 
                       index_label='date')  
data.index = data.index + pd.DateOffset(days=14)
data.iloc[cut:].to_csv(data_dir / 'uwe_cps_shift.csv', 
                       index_label='date')  

# Calculate growth rate for BLS published data
bls = (pd.read_csv(data_dir / 'uwe_main.csv', parse_dates=['date'])
         .set_index('date'))
blsgr = (bls.pct_change(4) * 100).loc['1989':]
blsgr.index = blsgr.index + pd.DateOffset(days=45)
blsgr.to_csv(data_dir / 'uwe_bls_gr.csv', index_label='date')
blsgr.iloc[-2:].to_csv(data_dir / 'uwe_bls_sh.csv', 
                       index_label='date')  

In [4]:
# Median wage text
d = {}
for i in [-1, -2, -5]:
    d[i] = {'dt': dtxt(bls.index[i])['qtr2'],
            'dt2': dtxt(bls.index[i])['qtr1'],
            'val': f'\${bls.p50uwe.iloc[i]:,.0f} per week'}
    
ltch = value_text(blsgr.p50uwe.iloc[-1], 'increase_of', 
                  time_str='nominal one-year ', threshold=0.1)
prch = value_text(blsgr.p50uwe.iloc[-2], 'increase_of', 
                  time_str='one-year ', threshold=0.1)
also = 'also ' if blsgr.p50uwe.iloc[-1].round(1) == blsgr.p50uwe.iloc[-2].round(1) else ''
cl = c_line('cyan!60!white')
text = (f'In {d[-1]["dt"]}, median usual earnings of full-time wage and '+
        f'salary workers are {d[-1]["val"]}, compared to {d[-5]["val"]} '+
        f'in {d[-5]["dt2"]}, {ltch} {cl}. In {d[-2]["dt2"]}, the median full-time '+
        f'worker receives {d[-2]["val"]}, {also}{prch}.')
write_txt(text_dir / 'uwe_median.txt', text)
print(text)

In the first quarter of 2023, median usual earnings of full-time wage and salary workers are \$1,100 per week, compared to \$1,037 per week in 2022 Q1, a nominal one-year increase of 6.1 percent (see {\color{cyan!60!white}\textbf{---}}). In 2022 Q4, the median full-time worker receives \$1,085 per week, a one-year increase of 7.4 percent.


In [5]:
d = {}
for i in [-1, -2, -3]:
    d[i] = {'dt': dtxt(data.dropna().index[i])['mon1'],
            'val': f'\${data.p50.dropna().iloc[i]:,.0f} per week',
            'gr': value_text(data['p50_gr'].dropna().iloc[i], 'increase_of'),
            'gr2': value_text(data['p50_gr'].dropna().iloc[i])}
    
avg = data.p50.dropna().rolling(3).mean().iloc[-1]
avgt = f'\${avg:,.0f} per week'
ch3m = value_text(data.p50_3M_gr.dropna().iloc[-1], 'increase_of', 
                  threshold=0.1)
color = 'violet!80!blue'
cl2 = c_line(color)    
cb = c_box(color)

prdt = (d[-2]['dt'][:-5] if d[-2]['dt'][-4:] == d[-1]['dt'][-4:] 
        else d[-2]['dt'])
pr2dt = (d[-3]['dt'][:-5] if d[-3]['dt'][-4:] == d[-1]['dt'][-4:] 
        else d[-3]['dt'])

text = (f'In {d[-1]["dt"]}, the median full-time worker receives '+
        f'{d[-1]["val"]}, following {d[-2]["val"]} in {d[-2]["dt"]} '+
        f'and {d[-3]["val"]} in {d[-3]["dt"]}. The average over '+
        f'these three months is {avgt}, {ch3m} over the same three '+
        f'months, one year prior {cl2}. \n\nMedian usual weekly '+
        f'earnings {d[-1]["gr2"]} over the year ending {d[-1]["dt"]} '+
        f'{cb}, following {d[-2]["gr"]} in {prdt}, and '+
        f'{d[-3]["gr"]} in {pr2dt}. ')
write_txt(text_dir / 'uwe_median_cps.txt', text)
print(text)

In March 2023, the median full-time worker receives \$1,098 per week, following \$1,123 per week in February 2023 and \$1,101 per week in January 2023. The average over these three months is \$1,107 per week, an increase of 5.8 percent over the same three months, one year prior (see {\color{violet!80!blue}\textbf{---}}). 

Median usual weekly earnings increased 5.1 percent over the year ending March 2023 (see \cbox{violet!80!blue}), following an increase of five percent in February, and an increase of 7.2 percent in January. 


In [6]:
# BLS published
d = {}
for i in [-1, -2, -5]:
    d[i] = {'dt': dtxt(bls.index[i])['qtr1'],
            'val': f'\${bls.p10uwe.iloc[i]:,.0f} per week'}
    
ltch = value_text(blsgr.p10uwe.iloc[-1], 'increase_of', 
                  time_str='nominal one-year ', threshold=0.1)
prch = value_text(blsgr.p10uwe.iloc[-2], threshold=0.1)
also = 'also ' if blsgr.p10uwe.iloc[-1].round(1) == blsgr.p10uwe.iloc[-2].round(1) else ''
cl = c_line('blue!65!black')
cl2 = c_line('lime!65!green!90!black')

# CPS-based
d2 = {}
for i in [-1, -2, -3]:
    d2[i] = {'dt': dtxt(data.dropna().index[i])['mon1'],
             'val': f'\${data.p10.dropna().iloc[i]:,.0f} per week',
             'val2': f'\${data.p10_3M.dropna().iloc[i]:,.0f} per week',             
             'gr': value_text(data['p10_gr'].dropna().iloc[i], 'increase_of'),
             'gr2': value_text(data['p10_gr'].dropna().iloc[i], threshold=0.1),
             'gr3': value_text(data['p10_3M_gr'].dropna().iloc[i], threshold=0.1),
             'gr4': value_text(data['p10_gr'].dropna().iloc[i], 'increase_of', 
                               time_str='one-year ', threshold=0.1),
             'gr5': value_text(data['p10_gr'].dropna().iloc[i], 'plain', 
                               threshold=0.1)}

moch2 = d2[-2]['gr5'] if d2[-1]['gr2'][:2] == d2[-2]['gr2'][:2] else d2[-2]['gr']
moch3 = (d2[-3]['gr5'] if (d2[-1]['gr2'][:2] == d2[-3]['gr2'][:2]) & 
 (d2[-2]['gr2'][:2] == d2[-3]['gr2'][:2]) else d2[-3]['gr'])
    
url = 'https://www.bls.gov/webapps/legacy/cpswktab5.htm'    
text = (f'BLS \href{{{url}}}{{report}} first decile usual earnings for '+
        f'full-time workers of {d[-1]["val"]} in {d[-1]["dt"]} and '+
        f'{d[-5]["val"]} in {d[-5]["dt"]}, {ltch} {cl}. Over the year ending '+
        f'{d[-2]["dt"]}, first decile usual weekly earnings {also}{prch}.\n\n'+
        'The more-volatile CPS-based monthly measure shows first decile '+
        f'usual earnings of {d2[-1]["val"]} in {d2[-1]["dt"]}, {d2[-2]["val"]} '+
        f'in {d2[-2]["dt"]}, and {d2[-3]["val"]} in {d2[-3]["dt"]}. The three-'+
        f'month average is {d2[-1]["val2"]}; first decile earnings '+
        f'{d2[-1]["gr3"]} over the same months, one-year prior {cl2}. By month, over '+
        f'the year ending {d2[-1]["dt"]}, first decile earnings {d2[-1]["gr2"]}, '+
        f'following {moch2} in {d2[-2]["dt"]}, and {moch3} in {d2[-3]["dt"]}.')
write_txt(text_dir / 'uwe_p10_basic.txt', text)
print(text)

BLS \href{https://www.bls.gov/webapps/legacy/cpswktab5.htm}{report} first decile usual earnings for full-time workers of \$574 per week in 2023 Q1 and \$531 per week in 2022 Q1, a nominal one-year increase of 8.1 percent (see {\color{blue!65!black}\textbf{---}}). Over the year ending 2022 Q4, first decile usual weekly earnings increased 9.8 percent.

The more-volatile CPS-based monthly measure shows first decile usual earnings of \$578 per week in March 2023, \$565 per week in February 2023, and \$581 per week in January 2023. The three-month average is \$575 per week; first decile earnings increased 7.6 percent over the same months, one-year prior (see {\color{lime!65!green!90!black}\textbf{---}}). By month, over the year ending March 2023, first decile earnings increased 7.7 percent, following 4.7 percent in February 2023, and 10.5 percent in January 2023.


In [7]:
srs = {'First decile': 'p10uwe', 'First quartile': 'p25uwe', 
       'Median': 'p50uwe', 'Third quartile': 'p75uwe', 
       'Ninth decile': 'p90uwe'}
df3 = (pd.read_csv(data_dir / 'uwe_main.csv', parse_dates=['date'])
        .set_index('date')).loc['2000':, srs.values()].dropna()

final = pd.DataFrame()
for i in [-1, -2, -3, -4, -5, -9, -13, -17, -21]:
    final[dtxt(df3.index[i])['qtr1']] = df3.pct_change(4).iloc[i] * 100

final.index = srs.keys()
final.round(1).to_csv(data_dir / 'wage_dist_bls.tex', sep='&', 
                      lineterminator='\\\ ', quotechar=' ')

final = pd.DataFrame()
for i in [-1, -2, -3, -4, -5, -9, -13, -17, -21]:
    final[dtxt(df3.index[i])['qtr1']] = df3.iloc[i]

final.index = srs.keys()
(final.round(0).astype('int').applymap('{:,.0f}'.format)
      .to_csv(data_dir / 'wage_dist_bls2.tex', sep='&', 
              lineterminator='\\\ ', quotechar=' '))

In [8]:
final.round(0).astype('int').applymap('{:,.0f}'.format)

,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,2021 Q1,2020 Q1,2019 Q1,2018 Q1
First decile,574,571,560,547,531,486,468,442,423
First quartile,739,736,724,710,701,657,630,605,589
Median,"1,100","1,085","1,070","1,041","1,037",989,957,905,881
Third quartile,"1,751","1,709","1,696","1,655","1,635","1,563","1,513","1,451","1,399"
Ninth decile,"2,718","2,584","2,583","2,561","2,512","2,424","2,320","2,265","2,155"
